In [2]:
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
import glob
import os
import pandas as pd

class ProgBar:

    def __init__(self, n_elements,int_str):
        
        import sys

        self.n_elements = n_elements
        self.progress = 0

        print(int_str)

        # initiallizing progress bar

        info = '{:.2f}% - {:d} of {:d}'.format(0,0,n_elements)

        formated_bar = ' '*int(50)

        sys.stdout.write("\r")

        sys.stdout.write('[%s] %s' % (formated_bar,info))

        sys.stdout.flush()

    def update(self,prog_info=None):
        
        import sys

        if prog_info == None:

            self.progress += 1

            percent = (self.progress)/self.n_elements * 100 / 2

            info = '{:.2f}% - {:d} of {:d}'.format(percent*2,self.progress,self.n_elements)

            formated_bar = '-'* int (percent) + ' '*int(50-percent)

            sys.stdout.write("\r")

            sys.stdout.write('[%s] %s' % (formated_bar,info))

            sys.stdout.flush()


        else:

            self.progress += 1

            percent = (self.progress)/self.n_elements * 100 / 2

            info = '{:.2f}% - {:d} of {:d} '.format(percent*2,self.progress,self.n_elements) + prog_info

            formated_bar = '-'* int (percent) + ' '*int(50-percent)

            sys.stdout.write("\r")

            sys.stdout.write('[%s] %s' % (formated_bar,info))

            sys.stdout.flush()


In [5]:
file_list = glob.glob('raw_data/*')
file_list.sort()


In [6]:
bar = ProgBar(len(file_list),"Reading and appending files...")

converted_files = glob.glob('pickle/*')

for file in file_list:

    file_name = file.split('/')[-1]

    convert_name = "pickle/" + file_name + ".pkl"

    if convert_name not in converted_files:

        data = pd.read_csv(file)

        pickle.dump(data.values,
                    open(convert_name, "wb"), 
                    )
                    
    bar.update()
    

Reading and appending files...
[--------------------------------------------------] 100.00% - 16 of 16

In [7]:
converted_files = glob.glob('pickle/*')

converted_files.sort()

converted_files

['pickle/18-Jan-2022-09.11.05.csv.pkl',
 'pickle/18-Jan-2022-09.48.05.csv.pkl',
 'pickle/18-Jan-2022-10.10.58.csv.pkl',
 'pickle/18-Jan-2022-10.30.43.csv.pkl',
 'pickle/18-Jan-2022-10.49.48.csv.pkl',
 'pickle/18-Jan-2022-11.15.56.csv.pkl',
 'pickle/18-Jan-2022-11.41.18.csv.pkl',
 'pickle/18-Jan-2022-12.04.21.csv.pkl',
 'pickle/18-Jan-2022-12.24.50.csv.pkl',
 'pickle/18-Jan-2022-12.45.51.csv.pkl',
 'pickle/18-Jan-2022-14.25.20.csv.pkl',
 'pickle/18-Jan-2022-15.01.43.csv.pkl',
 'pickle/18-Jan-2022-15.29.52.csv.pkl',
 'pickle/18-Jan-2022-15.57.26.csv.pkl',
 'pickle/18-Jan-2022-16.08.40.csv.pkl',
 'pickle/18-Jan-2022-16.20.45.csv.pkl']

In [9]:

percentages = [4]

count = 0

bar = ProgBar(len(converted_files)*len(percentages)*33,"Performing data scaling...")

divisions = glob.glob('pickle/*')

for percent in percentages:

    for i in range(33):

        n_good = 0
        n_bad = 0

        for f,file in enumerate(converted_files):

            data  = pickle.load(open(file,'rb'))

            aux = np.zeros((1,len(data[0])))

            data = np.vstack((data,aux))

            if f < 13:
                target = np.zeros(len(data)).reshape(-1,1)

                data = np.hstack((data,target))

            else:
                target = np.ones(len(data)).reshape(-1,1)

                data = np.hstack((data,target))

            n_measures = int(np.max(data[:, 1]))
            n_ids = len(np.unique(data[:,0]))-1

            expanded_dimensions = data.reshape(n_ids, n_measures, 9)

            _, fold = train_test_split(expanded_dimensions,
                                       test_size=percent)

            train_fold, test_fold = train_test_split(fold,
                                                     test_size=0.3)

            if f == 0:
                train_division = train_fold
                test_division = test_fold

            else:
                train_division = np.vstack((train_division,train_fold))
                test_division = np.vstack((test_division,test_fold))

            bar.update()


        pickle.dump(train_division,
                    open("train_divisions/division__{}__n_series__{}__{}__.pkl".format(count,percent,i), "wb"))
        
        pickle.dump(test_division,
                    open("test_divisions/division__{}__n_series__{}__{}__.pkl".format(count,percent,i), "wb"))
        
        count += 1



Performing data scaling...
[--------------------------------------------------] 100.00% - 528 of 528

In [4]:
target = np.ones(len(flattened_input))

target[:12758] = 0

pickle.dump(flattened_input,
            open("/media/thiago/ANGELICA/thiago/pickle/flatenned_input__1.pkl", "wb"), 
            protocol=4)

pickle.dump(flattened_input,
            open("/media/thiago/ANGELICA/thiago/pickle/flatenned_target__1.pkl", "wb"), 
            protocol=4)

NameError: name 'flattened_input' is not defined

In [ ]:
good_tools = pickle.load(open("good_tools.pkl", "rb"))

bad_tools = pickle.load(open("bad_tools.pkl", "rb"))


In [11]:
file_list = glob.glob("data_divisions/*")
file_list.sort()

In [7]:
for ii in range(int(len(file_list)/33)):

    for j in range(33):

        loc = ii*33 + j

        print(file_list[int(loc)])

        file = file_list[int(loc)]

        data = pickle.load(open(file, "rb"))